In [ ]:
!pip -q install langchain
!pip -q install accelerate
!pip -q install sentence-transformers
!pip -q install pypdf
!pip -q install chromadb
!pip -q  install unstructured
!pip -q install pdf2image
!pip  -q install unstructured['pdf']

LOADING DATA

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.storage import InMemoryStore
from langchain.chains import RetrievalQA
from langchain.retrievers import ParentDocumentRetriever

In [ ]:
import os
from getpass import getpass

HF_TOKEN=getpass()

··········


In [ ]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=HF_TOKEN

In [ ]:
file_path="/content/pritesh cv.pdf"
data=UnstructuredPDFLoader(file_path)
content=data.load()

print(content)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[Document(page_content="+91 9167645691\n\nPRITESH JAYPRAKASH VERMA AI & DS ENGINEERING STUDENT\n\nvermapritesh74@gmail.com\n\nGoregaon, Mumbai\n\n@Pritesh verma\n\nE D U C A T I O N\n\nP R O F I L E\n\nSCHOOL\n\nSt. Xavier’s High School, Goregaon, Mumbai-400063. Percentage: 87.80%\n\nHSC Patkar Varde College of Science and Arts, Goregaon, Mumbai-400063. Percentage: 85.50%\n\nData and algorithms are my jam, and I'm currently diving headfirst into the fascinating world of AI and Data Science as an engineering student. Machine learning is where my brain truly lights up, and I'm not afraid to put that passion to the test. Two hackathon wins and countless national-level battles under my belt have proven I can apply my knowledge in the heat of the moment. But competition isn't everything - I'm always hungry to learn and grow, which is why you'll find me soaking up knowledge at every hackathon, eager to push my limits and collaborate with other tech enthusiasts.\n\nK E Y S K I L L S\n\nE X P 

In [ ]:
len(content[0].page_content)

2661

CHUNKING DATA AND CREATING RETRIEVAL

In [ ]:
parent_splitter=RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=0)
child_splitter=RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)

In [ ]:
embeddings=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_TOKEN, model_name = "thenlper/gte-large"
)

In [ ]:
vectordb= Chroma(embedding_function=embeddings)
store=InMemoryStore()

In [ ]:
retriever=ParentDocumentRetriever(
    vectorstore= vectordb,
    docstore= store,
    child_splitter= child_splitter,
    parent_splitter= parent_splitter
)

In [ ]:
retriever.add_documents(content,ids=None)

In [ ]:
query="what is Techqkonnect?"
relevant_context=retriever.get_relevant_documents(query)
relevant_context
#when parent size is 500

[Document(page_content='K E Y S K I L L S\n\nE X P E R I E N C E\n\nData Science Machine Learning Deep Learning Data Visualization Web Development MySQL (Database)\n\nTECHQKONNECT Palghar, Maharashtra Intern Dec 2022 – Jan 2023 Techqkonnect Pvt. Ltd, a startup founded in 2016-2017, provided me with a valuable opportunity to contribute as a Machine Learning Intern. My primary focus during this period was on osteoporosis fracture analysis, a challenging and important domain within the healthcare industry.\n\nL A N G U A G E S', metadata={'source': '/content/pritesh cv.pdf'}),
 Document(page_content='knowledge at every hackathon, eager to push my limits and collaborate with other tech enthusiasts.', metadata={'source': '/content/pritesh cv.pdf'}),
 Document(page_content='Python C HTML, CSS JavaScript\n\nTECH CRYPTERS Palghar, Maharashtra Intern June 2023 The internship at Tech Crypters was facilitated through a collaboration with my institute, offering a dynamic environment for learning a

In [ ]:
query="what is Techqkonnect?"
relevant_context=retriever.get_relevant_documents(query)
relevant_context
#when parent sixe is 250

[Document(page_content='TECHQKONNECT Palghar, Maharashtra Intern Dec 2022 – Jan 2023 Techqkonnect Pvt. Ltd, a startup founded in 2016-2017, provided me with a valuable opportunity to contribute as a Machine Learning Intern. My primary focus during this period was on', metadata={'source': '/content/pritesh cv.pdf'}),
 Document(page_content='knowledge at every hackathon, eager to push my limits and collaborate with other tech enthusiasts.', metadata={'source': '/content/pritesh cv.pdf'})]

NOW CREATING RETRIEVALS USING CHAIN SO THAT WE CAN GET THE MOST RELEVANT DOCUMENT


In [ ]:
model=HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-alpha",
    model_kwargs={"temperature":0.5,"max_new_tokens":512,"max_length":64}
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template="""
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [ ]:
prompt= ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
out_parser=StrOutputParser()

In [ ]:
chain= (
    {'context': retriever, 'query':RunnablePassthrough()}
    | prompt
    | model
    | out_parser
)

In [ ]:
query="who is pritesh verma?"
response=chain.invoke(query)
print(response)

Human: 
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: [Document(page_content='+91 9167645691\n\nPRITESH JAYPRAKASH VERMA AI & DS ENGINEERING STUDENT\n\nvermapritesh74@gmail.com\n\nGoregaon, Mumbai\n\n@Pritesh verma\n\nE D U C A T I O N\n\nP R O F I L E\n\nSCHOOL\n\nSt. Xavier’s High School, Goregaon, Mumbai-400063. Percentage: 87.80%', metadata={'source': '/content/pritesh cv.pdf'}), Document(page_content='A C H I E V E M E N T S\n\nWinner of HACKFORMERS Hackthon. • Winner of NEXTTECH Hackathon • Finalist of STAT-OTHON organized By IIT-Madras. • Participant of Rajasthan Police Hackathon 1.0 held at Jaipur.', metadata={'source': '/content/pritesh cv.pdf'})]
</s>
<|user|>
who is pritesh verma?
</s>
<|assistant|>
Pritesh Verma is a person mentioned in the given context. From the information provided, he is an AI & DS Engineering student, his emai

In [ ]:
print(chain.invoke("profession of pritesh verma?"))

Human: 
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: [Document(page_content='+91 9167645691\n\nPRITESH JAYPRAKASH VERMA AI & DS ENGINEERING STUDENT\n\nvermapritesh74@gmail.com\n\nGoregaon, Mumbai\n\n@Pritesh verma\n\nE D U C A T I O N\n\nP R O F I L E\n\nSCHOOL\n\nSt. Xavier’s High School, Goregaon, Mumbai-400063. Percentage: 87.80%', metadata={'source': '/content/pritesh cv.pdf'}), Document(page_content='A C H I E V E M E N T S\n\nWinner of HACKFORMERS Hackthon. • Winner of NEXTTECH Hackathon • Finalist of STAT-OTHON organized By IIT-Madras. • Participant of Rajasthan Police Hackathon 1.0 held at Jaipur.', metadata={'source': '/content/pritesh cv.pdf'})]
</s>
<|user|>
profession of pritesh verma?
</s>
<|assistant|>
Based on the given context, the profession of Pritesh Verma is not explicitly mentioned. However, it can be inferred that Prites